## 安裝環境所需的套件與軟體

In [ ]:
!pip3 install pyttsx3 gtts edge_tts notion moviepy google-api-python-client google-auth-oauthlib google-auth-httplib2
!sudo apt update
!sudo apt install ffmpeg espeak libespeak1 imagemagick libmagick++-dev
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 3.9 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages

In [ ]:
!echo $PATH
!whereis convert

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
convert: /usr/bin/convert /usr/share/man/man1/convert.1.gz


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. 先建一資料夾放專案files


In [ ]:
import os
import datetime

def create_folder_with_today_date():
    # 取得今天的日期
    today_date = datetime.datetime.today().strftime('%Y%m%d')

    # 檢查資料夾是否已存在，如果不存在則建立
    folder_name = f"./{today_date}"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"專案資料夾 {folder_name} 已建立")
    else:
        print(f"專案資料夾 {folder_name} 已存在")
    return folder_name

# 呼叫函式以建立資料夾
project_folder = create_folder_with_today_date()

專案資料夾 ./20240624 已建立


## 2. 跟Notion要到當天的data: tuple = (’禱告主題’, ‘經文出處’, ‘經文文字’, ‘禱告詞’)


## 3. Edgetts產出語音檔


In [ ]:
import os
from datetime import datetime
from notion.client import NotionClient
from google.colab import userdata
import requests
def get_notion_data(today_date):
    today_date = today_date
    topic, verse, verse_text, prayer = (None, None, None, None)
    # 設定 Notion Token 和 database 的 ID
    token_v2 = userdata.get('NOTION_TOKEN')
    database_id = userdata.get('NOTION_DB_ID')
    r = requests.post(
      url=f"https://api.notion.com/v1/databases/{database_id}/query" ,
      headers={"Authorization": "Bearer " + token_v2, "Notion-Version": "2021-05-13"})
    data = r.json()
    # print(data)

    for item in data["results"]:
      # 检查是否存在"Date"属性，并且"name"为"NotFinished"
      properties = item.get("properties", {})
      d = properties.get("Date", {}).get("date",{})
      if d.get("start") == today_date:
        verse = properties.get("Verse", {}).get("rich_text", [])[0].get("text", {}).get("content")
        verse_text = properties.get("VerseText", {}).get("rich_text", [])[0].get("text", {}).get("content")
        prayer = properties.get("Prayer", {}).get("rich_text", [])[0].get("text", {}).get("content")
        topic = properties.get("Topic", {}).get("title", [])[0].get("text", {}).get("content")

        break
    return topic, verse, verse_text, prayer


# 呼叫函式並取得回傳的四個變數
today_date = datetime.today().strftime('%Y-%m-%d')
topic, verse, verse_text, prayer = get_notion_data(today_date)
print("Topic:", topic)
print("Verse:", verse)
print("Verse Text:", verse_text)
print("Prayer:", prayer)

Topic: None
Verse: None
Verse Text: None
Prayer: None


In [ ]:
import subprocess

def generate_voiceover(voice, text, file_name):
    # 將prayer插入到指令中
    cmd = f'edge-tts --rate=-20% --voice {voice} --text {text} --write-media {project_folder}/{file_name}.mp3'

    try:
        # 執行命令列指令
        subprocess.run(cmd, shell=True, check=True)
        print(f"成功產出聲音檔案: {today_date}/{file_name}.mp3")
    except subprocess.CalledProcessError as e:
        print("產出聲音檔案失敗:", e)

# 呼叫函式並傳遞 經文出處 變數
generate_voiceover("zh-CN-YunyangNeural", verse, "verse")

# 呼叫函式並傳遞  經文內容 變數
generate_voiceover("zh-CN-YunyangNeural", verse_text, "verse_text")

# 呼叫函式並傳遞prayer變數
generate_voiceover("zh-CN-YunyangNeural", prayer, "prayer")

成功產出聲音檔案: 2024-06-24/verse.mp3
成功產出聲音檔案: 2024-06-24/verse_text.mp3
成功產出聲音檔案: 2024-06-24/prayer.mp3


## 4. 從google drive隨機取得內容圖, 隨機取得背景樂, 選好當月的封面圖


In [ ]:
# Load the Drive helper and mount
from google.colab import drive
import os
import random
from datetime import datetime

def select_random_file(folder_path):
    # 列印資料夾內的所有檔案
    files = os.listdir(folder_path)

    # 隨機選取一個檔案並回傳其完整路徑位置
    random_file = random.choice(files)
    random_file_path = os.path.join(folder_path, random_file)
    return random_file_path

def get_cur_month_file(folder_path):
  # 取得當天所屬的年與月
  now = datetime.now()
  year_month = now.strftime("%Y%m")
  files = os.listdir(folder_path)

  # 比對資料夾中的檔案名稱
  for file in files:
      if file == f"{year_month}.jpg":
        return os.path.join(folder_path, file)
  return os.path.join(folder_path, "default.jpg")

# mount上去
# drive.mount('/content/drive')

# 呼叫函式隨機選內容圖片
content_img_folder_path = "/content/drive/MyDrive/Photos/PrayingBookstore/ContentPics"
content_image_path = select_random_file(content_img_folder_path)
print("隨機選取到的圖片檔案：", content_image_path)

# 呼叫函式隨機選背景音樂
music_folder_path = '/content/drive/MyDrive/Photos/PrayingBookstore/BGMusics'
bg_music_path = select_random_file(music_folder_path)
print("隨機選取到音樂片檔案：", bg_music_path)

# 呼叫函式取得該月的封面圖片
cover_img_folder_path = '/content/drive/MyDrive/Photos/PrayingBookstore/CoverPics'
cover_image_path = get_cur_month_file(cover_img_folder_path)
print("取得的封面圖片檔案：", cover_image_path)

隨機選取到的圖片檔案： /content/drive/MyDrive/Photos/PrayingBookstore/ContentPics/Default_christian_back_long_range_view_glory_light_sun_shine_p_0 (2).jpg
隨機選取到音樂片檔案： /content/drive/MyDrive/Photos/PrayingBookstore/BGMusics/The Five Books - I Think I Can Help You.mp3
取得的封面圖片檔案： /content/drive/MyDrive/Photos/PrayingBookstore/CoverPics/default.jpg


## 5. PIL開始製作封面圖


##6-1. 製作第一段片頭 標題, 經文出處, 經文影片產出



In [ ]:
from moviepy.editor import VideoClip, ImageClip, CompositeVideoClip, TextClip, AudioFileClip, concatenate_audioclips, concatenate_videoclips
from moviepy.video.fx.all import resize
from moviepy.config import change_settings
change_settings({"IMAGEMAGICK_BINARY": "/usr/bin/convert"})
TOPIC_WIDTH_MAX = 8
VERSE_WIDTH_MAX = 16
VERSET_WIDTH_MAX = 10

verse_audio_clip = AudioFileClip(f"{project_folder}/verse.mp3")
VOICE_TOPIC_VERSE_LEN = verse_audio_clip.duration

verse_text_audio_clip = AudioFileClip(f"{project_folder}/verse_text.mp3")
VOICE_TOPIC_VERSE_TEXT_LEN = verse_text_audio_clip.duration


def adjust_text_linechange(topic, verse, verse_text):
   # 針對topic 長度去做處理
   topic_len = len(topic)
   if topic_len <= TOPIC_WIDTH_MAX:
      pass
   elif topic_len > TOPIC_WIDTH_MAX and topic_len <= TOPIC_WIDTH_MAX * 2:
      topic = topic[:topic_len//2] + '\n' + topic[topic_len//2:]
   elif topic_len > 2 * TOPIC_WIDTH_MAX:
      topic = topic[:topic_len//3] + '\n' + topic[topic_len//3:2*topic_len//3] + '\n' + topic[2*topic_len//3:]

   # 針對verse 長度去做處理
   verse_len = len(verse)
   if verse_len <= VERSE_WIDTH_MAX:
      pass
   elif verse_len > VERSE_WIDTH_MAX and verse_len <= VERSE_WIDTH_MAX * 2:
      verse = verse[:verse_len/2] + '\n' + verse[verse_len/2:]
   else:
      verse = verse[:verse_len/3] + '\n' + verse[verse_len/3:2*verse_len/3] + '\n' + verse[2*verse_len/3:]

   # 針對verse text 長度去做處理
   verset_len = len(verse_text)
   if verset_len <= VERSET_WIDTH_MAX:
      pass
   elif verset_len > VERSET_WIDTH_MAX and verset_len <= VERSET_WIDTH_MAX * 2:
      verse_text = verse_text[:verset_len//2] + '\n' + verse_text[verset_len//2:]
   elif verset_len > VERSET_WIDTH_MAX * 2 and verset_len <= VERSET_WIDTH_MAX * 3:
      verse_text = verse_text[:verset_len//3] + '\n' + verse_text[verset_len//3:2*verset_len//3] + '\n' + verse_text[2*verset_len//3:]
   else:
      verse_text = verse_text[:verset_len//4] + '\n' + verse_text[verset_len//4:verset_len//2] + '\n' + verse_text[verset_len//2:3*verset_len//4] + '\n' + verse_text[3*verset_len//4:]

   return (topic, verse, verse_text)

def add_text_to_cover_image(image_path, text_list, font_path, verse_font_path):
    # 讀取圖片
    image_clip = ImageClip(image_path)
    image_clip = resize(image_clip,(1080,1920))

    # 設定透明度為50%
    image_clip = image_clip.set_opacity(0.5)

    # 計算圖片中心點
    center_x = image_clip.size[0] / 2
    center_y = image_clip.size[1] / 2
    print(center_x)
    print(center_y)

    # 計算文字位置
    width, height = image_clip.size


    text_clip_topic = (TextClip(text_list[0], fontsize=100, color='white', bg_color='transparent', font=font_path, align='center', method='label')
                     .set_position(('center', center_y/10), relative=False)
                     .set_duration(VOICE_TOPIC_VERSE_TEXT_LEN+VOICE_TOPIC_VERSE_LEN+4)
                     .set_start(0)
                     .set_end(VOICE_TOPIC_VERSE_TEXT_LEN+VOICE_TOPIC_VERSE_LEN+4)
                     .crossfadein(0.5)
                     .crossfadeout(0.5))
    text_clip_verse = (TextClip(text_list[1], fontsize=65, color='gold', bg_color='transparent', font=font_path, align='center', method='label')
                     .set_position(('center', 2*center_y/2.2), relative=False)
                     .set_duration(VOICE_TOPIC_VERSE_TEXT_LEN+VOICE_TOPIC_VERSE_LEN+4)
                     .set_start(1.5)
                     .set_end(VOICE_TOPIC_VERSE_TEXT_LEN+VOICE_TOPIC_VERSE_LEN+4)
                     .crossfadein(1)
                     .crossfadeout(0.5))
    text_clip_verse_text = (TextClip(text_list[2], fontsize=70, color='white', bg_color='transparent', font=verse_font_path, align='center', method='label')
                     .set_position(('center', 2.4*center_y/2), relative=False)
                     .set_duration(VOICE_TOPIC_VERSE_TEXT_LEN+VOICE_TOPIC_VERSE_LEN+4)
                     .set_start(2)
                     .set_end(VOICE_TOPIC_VERSE_TEXT_LEN+VOICE_TOPIC_VERSE_LEN+4)
                     .crossfadein(1)
                     .crossfadeout(0.5))

    text_clips = [text_clip_topic, text_clip_verse, text_clip_verse_text]

    # 添加外框
    bordered_clip = resize(image_clip, width=image_clip.w+10, height=image_clip.h+10).margin(5, color=(123, 123, 22))

    # 讀取聲音檔案
    three_secs_silence = AudioFileClip(filename=f"/content/drive/MyDrive/Photos/PrayingBookstore/3_secs_silence.mp3")
    verse_audio = AudioFileClip(filename=f"{project_folder}/verse.mp3")
    verse_text_audio = AudioFileClip(filename=f"{project_folder}/verse_text.mp3")
    concatenated_audio = concatenate_audioclips([three_secs_silence, verse_audio, verse_text_audio])

    # 合成影片

    final_clip = CompositeVideoClip([bordered_clip] + text_clips).set_duration(concatenated_audio.duration+2)
    final_clip.audio = concatenated_audio
    # final_clip.write_videofile(filename=f"{project_folder}/head.mp4", fps=30, codec='libx264')

    return final_clip


# 呼叫函式
font_path = '/content/drive/MyDrive/Photos/PrayingBookstore/Fonts/NotoSerifTC-Black.otf'
verse_font_path = '/content/drive/MyDrive/Photos/PrayingBookstore/Fonts/NotoSerifTC-Light.otf'

cover_video_clip = add_text_to_cover_image(cover_image_path, adjust_text_linechange(topic, verse, verse_text), font_path, verse_font_path)

TypeError: object of type 'NoneType' has no len()

## 6-2. 內容圖 & 動態字體顯示 影片產出


In [ ]:


prayer_audio_clip = AudioFileClip(f"{project_folder}/prayer.mp3")
VOICE_PRAYER_LEN = prayer_audio_clip.duration

def split_str_to_multi_lines(text, max_chars) -> list:
  splitted_list = []

  cnt = 0
  index = 0
  for i,t in enumerate(text):
    cnt += 1
    if t == '。' or  cnt == max_chars:
      splitted_list.append(text[index:i+1])
      index = i+1
      cnt = 0
      if t == '。':
        splitted_list.append('\n')
  if index != len(text):
    splitted_list(text[index:])

  return splitted_list

def add_text_to_content_image(image_path, text_list, font_path):
    # 讀取圖片
    image_clip = ImageClip(image_path)
    image_clip = resize(image_clip,(1080,1920))

    # 設定透明度為50%
    image_clip = image_clip.set_opacity(0.5)

    # 計算圖片中心點
    center_x = image_clip.size[0] / 2
    center_y = image_clip.size[1] / 2
    print(center_x)
    print(center_y)
    len_y = image_clip.size[1]

    t_list = []
    for i, t in enumerate(text_list):
      text_clip = (TextClip(text_list[i], fontsize=70, color='white', bg_color='transparent', font=font_path, align='center', method='label')
                  .set_position((10, (i+1.2)*len_y/20), relative=False)
                  .set_duration(VOICE_PRAYER_LEN+6)
                  .set_start(0+(i*0.2))
                  .set_end(VOICE_PRAYER_LEN+6)
                  .crossfadein(0.2)
                  .crossfadeout(0.5))
      t_list.append(text_clip)

    # 添加外框
    # bordered_clip = resize(image_clip, width=image_clip.w+10, height=image_clip.h+10).margin(5, color=(123, 123, 22))

    # 讀取聲音檔案
    three_secs_silence = AudioFileClip(filename=f"/content/drive/MyDrive/Photos/PrayingBookstore/3_secs_silence.mp3")
    prayer_audio = AudioFileClip(filename=f"{project_folder}/prayer.mp3")
    concatenated_audio = concatenate_audioclips([three_secs_silence, prayer_audio])

    # 合成影片
    final_clip = CompositeVideoClip([image_clip] + t_list, size=(1080,1920)).set_duration(concatenated_audio.duration + 3)
    final_clip.audio = concatenated_audio
    # final_clip.write_videofile(filename=f"{project_folder}/content.mp4", fps=30, codec='libx264')

    return final_clip

verse_text_font_path = '/content/drive/MyDrive/Photos/PrayingBookstore/Fonts/Iansui-Regular.ttf'
prayer_list = split_str_to_multi_lines(prayer, 12)
content_video_clip = add_text_to_content_image(content_image_path, prayer_list, verse_text_font_path)


## 6-3. 將片頭影片與內容影片串接並配上背景音樂


In [ ]:
from moviepy.audio.fx.all import volumex
from moviepy.editor import CompositeAudioClip

final_clip = concatenate_videoclips([cover_video_clip, content_video_clip],method='compose').set_duration(cover_video_clip.duration + content_video_clip.duration)
final_clip = resize(final_clip, (1080, 1920))

bg_music_clip = AudioFileClip(filename=bg_music_path).set_duration(cover_video_clip.duration + content_video_clip.duration)
bg_music_clip = volumex(bg_music_clip, 0.05)

new_audioclip = CompositeAudioClip([final_clip.audio, bg_music_clip])
final_clip.audio = new_audioclip
final_clip.write_videofile(filename=f"{project_folder}/final.mp4", fps=30, codec='libx264')

##   6-4. **產出傳送至google** drive

In [ ]:
import shutil

# 產生影片標題例子： 天天禱告-20240321書房-為全球疫情結束與疫情受影響者的康復禱告.mp4
now = datetime.now()
yr_mon_date = now.strftime("%y%m%d")
video_name = f"{yr_mon_date}天天進入禱告書房-我們一起{topic}.mp4"

# 確認是否真的有final.mp4
if not os.path.exists(f'{project_folder}/final.mp4'):
  print('Did not find the final product video!!')
else:
  print(shutil.move(f"{project_folder}/final.mp4", f"/content/drive/MyDrive/Photos/PrayingBookstore/Artifacts/{video_name}")) # 將他移動到Google Drive中

# 7. 設定一個檢查機制確認
  1. Drive確實有備份影片檔案了
  2. 該筆在Notion中標記為完成

In [ ]:
# 確認drive中有這個檔案
if not os.path.exists(f"/content/drive/MyDrive/Photos/PrayingBookstore/Artifacts/{video_name}"):
  print('Did not find the final product video in google drive!!')
else:
  # 在Notion中把他的完成checkbox 打勾
  print('todo')